In [ ]:
!pip install -q 'git+https://github.com/drscook/voting_predictor'

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.1 MB 4.8 MB/s 
     |████████████████████████████████| 16.6 MB 185 kB/s 
     |████████████████████████████████| 7.8 MB 68.5 MB/s 
     |████████████████████████████████| 1.6 MB 57.0 MB/s 
     |████████████████████████████████| 43 kB 1.6 MB/s 
     |████████████████████████████████| 132 kB 63.0 MB/s 


In [1]:
%reload_ext google.colab.data_table
%reload_ext autotime
%reload_ext autoreload
%autoreload 2

root_path = '/content/drive/'
local = False
local = True
if local:
    import sys, google.colab
    google.colab.drive.mount(root_path)
    sys.path.insert(0, root_path + 'MyDrive/helpers')
    sys.path.insert(0, root_path + 'MyDrive/voting_predictor')
    root_path += 'MyDrive/gerrymandering/2022-12/'
else:
    !pip install -q --force-reinstall --no-deps 'git+https://github.com/drscook/helpers' 'git+https://github.com/drscook/voting_predictor'

from helpers.common_imports import *
from helpers import utilities as ut
import voting_predictor as vp

self = vp.Redistricter(
    census_api_key = 'dccb7bb4b7df5dff59d2d99c859016f973197e4e',
    bg_project_id = 'redistricting-361203',
    state = 'RI',
    root_path = root_path,
)

self.get_crosswalks()#overwrite=True)
self.get_assignments()#overwrite=True)
self.get_shapes()#overwrite=True)
tbl, df = self.get_pl(overwrite=True)
# self.combine()
# vp.test.f(5)
# ut.

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
crosswalks.RI ... fetching ... Elapsed time: 4.7642 seconds
assignments.RI2020 ... fetching ... Elapsed time: 4.1164 seconds
shapes.RI2020 ... Elapsed time: 0.0973 seconds
pl.RI2020 ... fetching ... 

,name,p2_001n,p4_001n,p2_005n,p4_005n,p2_002n,p4_002n,state,county,tract,block
index,,,,,,,,,,,
0,"Block 2011, Block Group 2, Census Tract 207.02...",8,6,5,3,0,0,44,3,20702,2011
1,"Block 1047, Block Group 1, Census Tract 209.01...",24,22,20,20,1,0,44,3,20901,1047
2,"Block 3003, Block Group 3, Census Tract 209.01...",36,33,36,33,0,0,44,3,20901,3003


,name,p2_001n,p4_001n,p2_005n,p4_005n,p2_002n,p4_002n,state,county,tract,block,year,block2020
index,,,,,,,,,,,,,
0,"Block 2011, Block Group 2, Census Tract 207.02...",8,6,5,3,0,0,44,3,20702,2011,2020,440030207022011
1,"Block 1047, Block Group 1, Census Tract 209.01...",24,22,20,20,1,0,44,3,20901,1047,2020,440030209011047
2,"Block 3003, Block Group 3, Census Tract 209.01...",36,33,36,33,0,0,44,3,20901,3003,2020,440030209013003


Elapsed time: 15.8279 seconds
